Mass of Aveosaurs - should be randomly selected for each individual (varies from 50 to 250 kgs)

Initiate positions randomly (but within the screen)

Initiate velocities (as 2D vectors) with magnitudes randomly selected between -30 to 30 kmph

radius of knowledge - 80m

begin with 100 creatures

leave out the constants (for force laws) as settable

In [ ]:
import warnings
warnings.filterwarnings("ignore")

import numpy as np

class Aveosaurs():

    def __init__(self, x, y, width, height):
        self.position = np.array([x, y])
        vec = (np.random.rand(2) - 0.5)*10
        self.velocity = np.array([*vec])

        vec = (np.random.rand(2) - 0.5)/2
        self.acceleration = np.array([*vec])
        self.max_force = 0.3
        self.max_speed = 5
        self.perception = 100

        self.width = width
        self.height = height



    def update(self):
        self.position += self.velocity
        self.velocity += self.acceleration
        #limit
        if np.linalg.norm(self.velocity) > self.max_speed:
            self.velocity = self.velocity / np.linalg.norm(self.velocity) * self.max_speed

        self.acceleration =np.array([*np.zeros(2)])

    def show(self):
        stroke(255)

        circle((self.position.x, self.position.y), radius=10)


    def apply_behaviour(self, boids):
        alignment = self.align(boids)
        cohesion = self.cohesion(boids)
        separation = self.separation(boids)

        self.acceleration += alignment
        self.acceleration += cohesion
        self.acceleration += separation

    def edges(self):
        if self.position.x > self.width:
            self.position.x = 0
        elif self.position.x < 0:
            self.position.x = self.width

        if self.position.y > self.height:
            self.position.y = 0
        elif self.position.y < 0:
            self.position.y = self.height

    def align(self, boids):
        steering = np.array([*np.zeros(2)])
        total = 0
        avg_vector = Vector(*np.zeros(2))
        for boid in boids:
            if np.linalg.norm(boid.position - self.position) < self.perception:
                avg_vector += boid.velocity
                total += 1
        if total > 0:
            avg_vector /= total
            avg_vector = np.array(*avg_vector)
            avg_vector = (avg_vector / np.linalg.norm(avg_vector)) * self.max_speed # Why?
            steering = avg_vector - self.velocity

        return steering

    def cohesion(self, boids):
        steering = np.array([*np.zeros(2)])
        total = 0
        center_of_mass = np.array([*np.zeros(2)])
        for boid in boids:
            if np.linalg.norm(boid.position - self.position) < self.perception:
                center_of_mass += boid.position
                total += 1
        if total > 0:
            center_of_mass /= total
            center_of_mass = np.array([*center_of_mass])
            vec_to_com = center_of_mass - self.position
            if np.linalg.norm(vec_to_com) > 0:
                vec_to_com = (vec_to_com / np.linalg.norm(vec_to_com)) * self.max_speed # Why?
            steering = vec_to_com - self.velocity
            if np.linalg.norm(steering) > self.max_force:
                steering = (steering /np.linalg.norm(steering)) * self.max_force

        return steering

    def separation(self, boids):
        steering = np.array([*np.zeros(2)])
        total = 0
        avg_vector = np.array([*np.zeros(2)])
        for boid in boids:
            distance = np.linalg.norm(boid.position - self.position)
            if self.position != boid.position and distance < self.perception:
                diff = self.position - boid.position
                diff /= distance
                avg_vector += diff
                total += 1
        if total > 0:
            avg_vector /= total
            avg_vector = np.array([*avg_vector])
            if np.linalg.norm(steering) > 0:
                avg_vector = (avg_vector / np.linalg.norm(steering)) * self.max_speed
            steering = avg_vector - self.velocity
            if np.linalg.norm(steering) > self.max_force:
                steering = (steering /np.linalg.norm(steering)) * self.max_force

        return steering
    
width = 1000
height = 1000

flock = [Aveosaurs(*np.random.rand(2)*1000, width, height) for _ in range(50)]